# Import libraries

In [15]:
import pandas as pd
import os

# Read data

In [16]:
path ='../data/car_price.csv'
df = pd.read_csv(path)
df.head()

,Unnamed: 0,car_name,car_prices_in_rupee,kms_driven,fuel_type,transmission,ownership,manufacture,engine,Seats
0,0,Jeep Compass 2.0 Longitude Option BSIV,10.03 Lakh,"86,226 kms",Diesel,Manual,1st Owner,2017,1956 cc,5 Seats
1,1,Renault Duster RXZ Turbo CVT,12.83 Lakh,"13,248 kms",Petrol,Automatic,1st Owner,2021,1330 cc,5 Seats
2,2,Toyota Camry 2.5 G,16.40 Lakh,"60,343 kms",Petrol,Automatic,1st Owner,2016,2494 cc,5 Seats
3,3,Honda Jazz VX CVT,7.77 Lakh,"26,696 kms",Petrol,Automatic,1st Owner,2018,1199 cc,5 Seats
4,4,Volkswagen Polo 1.2 MPI Highline,5.15 Lakh,"69,414 kms",Petrol,Manual,1st Owner,2016,1199 cc,5 Seats


# Describe data


In [19]:
def describle_data(df):
    print('Shape of data: ', df.shape)
    print('Columns of data: ', df.columns)
    print('Data types of data: ', df.dtypes)
    print('Missing values of data: ', df.isnull().sum())
    print('Describe data: ', df.describe())
    print('Info data: ', df.info())
describle_data(df)

Shape of data:  (5512, 10)
Columns of data:  Index(['Unnamed: 0', 'car_name', 'car_prices_in_rupee', 'kms_driven',
       'fuel_type', 'transmission', 'ownership', 'manufacture', 'engine',
       'Seats'],
      dtype='object')
Data types of data:  Unnamed: 0              int64
car_name               object
car_prices_in_rupee    object
kms_driven             object
fuel_type              object
transmission           object
ownership              object
manufacture             int64
engine                 object
Seats                  object
dtype: object
Missing values of data:  Unnamed: 0             0
car_name               0
car_prices_in_rupee    0
kms_driven             0
fuel_type              0
transmission           0
ownership              0
manufacture            0
engine                 0
Seats                  0
dtype: int64
Describe data:          Unnamed: 0  manufacture
count  5512.000000  5512.000000
mean   2755.500000  2015.455552
std    1591.321673     3.927974
min  